In [2]:
# some helpers

def get_html(url):
    res = requests.get(url,headers= {"User-Agent":"Mozilla Firefox Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0."})
    tree = html.fromstring(res.content)
    return tree

def get_products(urls):
    
    rows = []
    
    for url in urls:
        
        tree = get_html(url)
        [rows.append(etree.tostring(elem)) for elem in tree.cssselect('.product')]

        try:
            next_page = tree.cssselect('.next')[0].xpath('.//@href')[0]
            if next_page not in urls:
                urls.append(next_page)
        except:
            pass
        
    return rows

def process_products(data):
    
    df = pd.DataFrame()

    for product in data:

        prod = html.fromstring(product)

        prod_id = list(set(prod.xpath('.//a/@data-product_id')))[0]
        prod_sku = list(set(prod.xpath('.//a/@data-product_sku')))[0]
        prod_title = prod.xpath('.//h3/a/text()')[0]
        prod_url = prod.xpath('.//h3/a/@href')[0]
        price = prod.cssselect('.price')[0].xpath('.//text()')[1]

        tempdf = pd.DataFrame({
            'id':prod_id,
            'sku':prod_sku,
            'title':prod_title,
            'price':price,        
            'url':prod_url,
            'image':''.join(prod.xpath('.//img/@data-src')),
        }, index=[0])

        df = df.append(tempdf, sort=False)    
    
    return df

In [44]:
import requests, json
from lxml import html,etree
import pandas as pd
import datetime as dt

In [5]:
# prep seed urls

tree = get_html("https://amazonpets.ae/product-category/dog-supplies/")

links = [link for link in tree.xpath('.//a/@href') if '/cat-supplies/' in link or '/dog-supplies/' in link and 'page' not in link]
links = [link for link in links if link.endswith('/cat-supplies/') == False and link.endswith('/dog-supplies/') == False]
links = [link + "?orderby=price" for link in links]

In [6]:
len(links)

54

In [7]:
finaldf = pd.DataFrame()

for i,link in enumerate(links):
    urls = [link]
    
    data = get_products(urls)
    df = process_products(data)
    df['source'] = link
    
    finaldf = finaldf.append(df, sort=False)
    
    print(i, link, len(finaldf))

0 https://amazonpets.ae/product-category/cat-supplies/wet-food/?orderby=price 67
1 https://amazonpets.ae/product-category/cat-supplies/dry-food/?orderby=price 218
2 https://amazonpets.ae/product-category/cat-supplies/cat-accessories/?orderby=price 227
3 https://amazonpets.ae/product-category/cat-supplies/bowls-feeders/?orderby=price 253
4 https://amazonpets.ae/product-category/cat-supplies/carriers-containment/?orderby=price 283
5 https://amazonpets.ae/product-category/cat-supplies/furniture-scratchers/?orderby=price 350
6 https://amazonpets.ae/product-category/cat-supplies/grooming-bathing-cat-supplies/?orderby=price 371
7 https://amazonpets.ae/product-category/cat-supplies/litter/?orderby=price 427
8 https://amazonpets.ae/product-category/cat-supplies/litter-boxes-accessories/?orderby=price 481
9 https://amazonpets.ae/product-category/cat-supplies/cat-toys/?orderby=price 647
10 https://amazonpets.ae/product-category/cat-supplies/cat-collars-harnesses-and-leashes/?orderby=price 648
11

### Some checks, post-processing

In [42]:
finaldf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3253 entries, 0 to 0
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            3253 non-null   object 
 1   sku           3253 non-null   object 
 2   title         3253 non-null   object 
 3   price         3253 non-null   float64
 4   url           3253 non-null   object 
 5   image         3253 non-null   object 
 6   source        3253 non-null   object 
 7   category      3253 non-null   object 
 8   sub_category  3253 non-null   object 
dtypes: float64(1), object(8)
memory usage: 318.7+ KB


In [43]:
finaldf.head()

,id,sku,title,price,url,image,source,category,sub_category
0,17531,8005852941053,SCHESIR CAT CAN JELLY SENIOR CHICKEN FILLET/AL...,2.4,https://amazonpets.ae/product/schesir-cat-can-...,https://amazonpets.ae/wp-content/uploads/2020/...,https://amazonpets.ae/product-category/cat-sup...,cat-supplies,wet-food
0,13487,2170430037,Wellness Core Tender Cuts Ch/ChLvr Cat 85g,6.0,https://amazonpets.ae/product/wellness-core-te...,https://amazonpets.ae/wp-content/uploads/2020/...,https://amazonpets.ae/product-category/cat-sup...,cat-supplies,wet-food
0,13489,2170430038,Wellness Core Tender Cuts Salm/Tuna Cat 85g,6.0,https://amazonpets.ae/product/wellness-core-te...,https://amazonpets.ae/wp-content/uploads/2020/...,https://amazonpets.ae/product-category/cat-sup...,cat-supplies,wet-food
0,17495,8005852751010,SCHESIR CAT POUCH TUNA 100GM (C560),6.0,https://amazonpets.ae/product/scheair-cat-pouc...,https://amazonpets.ae/wp-content/uploads/2020/...,https://amazonpets.ae/product-category/cat-sup...,cat-supplies,wet-food
0,17591,8005852751034,SCHESIR CAT POUCH CHICKEN FILLET 100GM (C562),6.0,https://amazonpets.ae/product/schesir-cat-pouc...,https://amazonpets.ae/wp-content/uploads/2020/...,https://amazonpets.ae/product-category/cat-sup...,cat-supplies,wet-food


In [37]:
finaldf['category'] = finaldf['source'].apply(lambda x: x.split('?')[0].split('/')[-3])
finaldf['sub_category'] = finaldf['source'].apply(lambda x: x.split('?')[0].split('/')[-2])
finaldf['price'] = finaldf['price'].apply(lambda x: x.replace(',',''))
finaldf['price'] = finaldf['price'].astype(float)

In [38]:
finaldf.head(2)

,id,sku,title,price,url,image,source,category,sub_category
0,17531,8005852941053,SCHESIR CAT CAN JELLY SENIOR CHICKEN FILLET/AL...,2.4,https://amazonpets.ae/product/schesir-cat-can-...,https://amazonpets.ae/wp-content/uploads/2020/...,https://amazonpets.ae/product-category/cat-sup...,cat-supplies,wet-food
0,13487,2170430037,Wellness Core Tender Cuts Ch/ChLvr Cat 85g,6.0,https://amazonpets.ae/product/wellness-core-te...,https://amazonpets.ae/wp-content/uploads/2020/...,https://amazonpets.ae/product-category/cat-sup...,cat-supplies,wet-food


In [41]:
finaldf.groupby(['category','sub_category']).agg({
    'id':'count',
    'price':['min','max','mean']
})

id  price           \
                                               count    min      max   
category     sub_category                                              
cat-supplies bowls-feeders                        52  18.00   385.24   
             carriers-containment                 60  28.10   900.00   
             cat-accessories                      18  25.00   765.24   
             cat-collars-harnesses-and-leashes     2  45.00    45.00   
             cat-toys                            332   0.00   595.24   
             dog-health-wellness                   1  27.00    27.00   
             dry-food                            302   5.24   403.33   
             furniture-scratchers                134  25.00  1500.00   
             grooming-bathing-cat-supplies        42  12.38   199.00   
             litter                              112  11.90   799.05   
             litter-boxes-accessories            108  10.00   799.05   
             wet-food                            134   2.40   120.00   
dog-supplies auto-feeders-auto-dispensers         18  25.00   359.00   
             beds-cushions                       358  24.00   999.00   
             bottles                              10  16.80  3150.00   
             bowls-plates                        150   8.40   277.00   
             dog-accessories                      18  50.00  1513.33   
             dog-carriers-containment             46  99.00  1119.00   
             dog-collars-harnesses-and-leashes   326   0.00   357.00   
             dog-litter-boxes-accessories          4  50.00   105.00   
             dog-toys                             92  12.00   595.24   
             dog-treats                            8  23.81    33.34   
             dry-food-dog-supplies               358   9.05   459.05   
             fountains                             8  56.70   249.90   
             grooming-bathing-dog-supplies       464  12.60   619.00   
             storages-containers                   6  42.00    71.40   
             water-dispensers-food-dispensers     24  53.37   151.20   
             wet-food-dog-supplies                66   7.20    97.14   

                                                            
                                                      mean  
category     sub_category                                   
cat-supplies bowls-feeders                      165.354615  
             carriers-containment               215.620333  
             cat-accessories                    509.382222  
             cat-collars-harnesses-and-leashes   45.000000  
             cat-toys                            45.092651  
             dog-health-wellness                 27.000000  
             dry-food                            84.150927  
             furniture-scratchers               278.228209  
             grooming-bathing-cat-supplies       92.303810  
             litter                              63.171250  
             litter-boxes-accessories            91.955370  
             wet-food                            12.708806  
dog-supplies auto-feeders-auto-dispensers       121.010000  
             beds-cushions                      160.686089  
             bottles                            647.220000  
             bowls-plates                        38.235467  
             dog-accessories                    741.530000  
             dog-carriers-containment           337.543478  
             dog-collars-harnesses-and-leashes   67.481595  
             dog-litter-boxes-accessories        77.500000  
             dog-toys                            56.191739  
             dog-treats                          26.430000  
             dry-food-dog-supplies              122.262849  
             fountains                          176.612500  
             grooming-bathing-dog-supplies       73.525862  
             storages-containers                 60.900000  
             water-dispensers-food-dispensers   

In [49]:
finaldf.to_csv("amazonpets_products_{}.csv".format(int(dt.datetime.now().timestamp())), index=False)